Data Load

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
from google.colab import files
uploaded = files.upload()

In [ ]:
!mkdir data_store

In [ ]:
import os

path = '/content/drive/MyDrive/animal_data/train/'
os.listdir(path)[:5]

['cat.0.jpg', 'cat.5.jpg', 'cat.13.jpg', 'cat.12.jpg', 'cat.1.jpg']

라벨과 id로 나눔
(cat, dog로 라벨을 나눈다)

In [ ]:
full_name = os.listdir(path)
labels = [each.split('.')[0] for each in full_name]
file_id = [each.split('.')[1] for each in full_name]

In [ ]:
set(labels), len(file_id)

데이터 보기

In [ ]:
import random
import matplotlib.pyplot as plt
import matplotlib.image as mpimg

sample = random.choice(full_name)
image = mpimg.imread(path + sample)
plt.imshow(image)
plt.title(sample)
plt.show()

사진의 크기

In [ ]:
sample = random.choice(full_name)
image = mpimg.imread(path + sample)
print(image.shape)
sample = random.choice(full_name)
image = mpimg.imread(path + sample)
print(image.shape)

In [ ]:
from skimage.transform import resize
resized = resize(image, (128, 128, 3))

fig, axes = plt.subplots(1, 2, figsize=(8, 4))
ax = axes.ravel()

ax[0].imshow(image, cmap = plt.cm.gray)
ax[0].set_title('image')
ax[1].imshow(resized, cmap = plt.cm.gray)
ax[1].set_title('Resized')

fig.tight_layout()
plt.show()

Resized(128 * 128 * 3)


In [ ]:
from skimage.color import rgb2gray
import numpy as np

images = []
bar_total = full_name
for file in bar_total:
    image = mpimg.imread(path + file)
    images.append(resize(image, (128, 128, 3)))
images = np.array(images)

In [ ]:
images.shape, labels[:3]

Label 인코딩

In [ ]:
from sklearn.preprocessing import LabelEncoder

encoder = LabelEncoder()
encoder.fit(labels)
labels_encoded = encoder.transform(labels)
labels_encoded[:3], encoder.classes_

Data 나누기

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(images, labels_encoded, test_size = 0.2, random_state = 13, stratify = labels_encoded)

X_train.shape, X_test.shape

이미지 확인

In [ ]:
samples = random.choices(population = range(0,20000),k=8)

plt.figure(figsize = (14,12))
for idx, n in enumerate(samples):
    plt.subplot(2,4, idx+1)
    plt.imshow(X_train[n], cmap = 'Greys', interpolation = 'nearest')
    plt.title(y_train[n])

plt.tight_layout()
plt.show()

CNN 모델 생성

In [ ]:
from tensorflow.keras import layers, models

model = models.Sequential([
    layers.Conv2D(32, (3, 3), activation='relu', input_shape = (128, 128, 3)),
    layers.MaxPooling2D(pool_size=(2, 2), strides=(2, 2)),
    layers.Dropout(0.25),

    layers.Conv2D(64, (3, 3), activation='relu', padding='same'),
    layers.MaxPooling2D(pool_size=(2, 2)),
    layers.Dropout(0.25),

    layers.Conv2D(64, (3, 3), activation='relu', padding='same'),
    layers.MaxPooling2D(pool_size=(2, 2)),
    layers.Dropout(0.25),

    layers.Flatten(),
    layers.Dense(512, activation='relu'),
    layers.Dropout(0.25),
    layers.Dense(2, activation='softmax')
])

model.summary()

Fit

In [ ]:
import time
model.compile(
    optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])


start_time = time.time()
hist = model.fit(X_train.reshape(20000, 128, 128, 3), y_train, epochs = 5, verbose=1, validation_data= (X_test.reshape(5000, 128,128,3), y_test))

print(f'Fit Time : {time.time() - start_time}')

학습 상황

In [ ]:
plot_target = ['loss', 'val_loss', 'accuracy', 'val_accuracy']
plt.figure(figsize = (12,8))

for each in plot_target:
    plt.plot(hist.history[each], label = each)

plt.legend()
plt.grid()
plt.show()

Accuracy

In [ ]:
score = model.evaluate(X_test, y_test)
print(f'Test Loss : {score[0]}')
print(f'Test Accuracy  : {score[1]}')

Predict

In [ ]:
predicted_result = model.predict(X_test)
predicted_labels = np.argmax(predicted_result,  axis=1)
predicted_labels

In [ ]:
print("array([0, 1, 0, ..., 0, 1, 1])")

array([0, 1, 0, ..., 0, 1, 1])


틀린데이터 확인

In [ ]:
wrong_result = []
for n in range(0, len(y_test)):
    if predicted_labels[n] != y_test[n]:
        wrong_result.append(n)

len(wrong_result)

틀린 데이터 중 16개만 그려보기

In [ ]:
import random

samples = random.choices(population=wrong_result, k=16)

plt.figure(figsize=(14, 12))

for idx, n in enumerate(samples):
    plt.subplot(4, 4, idx + 1)
    plt.imshow(X_test[n].reshape(128, 128, 3),
               cmap='Greys', interpolation='nearest')
    plt.title('Label ' + str(y_test[n]) +
              ', Predict ' + str(predicted_labels[n]))
    plt.axis('off')

plt.show()

이미지 하나 넣고 확인(개인적 사진)

In [ ]:
import cv2
import matplotlib.pyplot as plt

image_bgr = cv2.imread("./data/dogs-vs-cats/cat_test.jpeg")
test_image = cv2.cvtColor(image_bgr, cv2.COLOR_BGR2RGB)
test_image = resize(test_image, (128, 128, 3))
plt.imshow(test_image)
plt.title('cat')
plt.show()

예측

In [ ]:
if np.argmax(model.predict(test_image.reshape(1, 128, 128, 3))) == 0:
    print('Cat')
else :
    print('Dos')